In [ ]:
import sys
import pandas as pd
from lxml import etree
import re
from unidecode import unidecode

print(sys.executable)

# Preprocessing

1. extration des articles à lier et des features utiles dans df
2. chargement du référentiel des communes (COG 2011) dans le df main_insee_commune

In [ ]:
# Extraction des features de chaque article à lier dans un df

xml_file = '../../data/PO_t11/PO_t11.xml'
xml_tree = etree.parse(xml_file)

data = []

i = 0 # compter les articles

for article in xml_tree.xpath('//article'):
    i+=1
    # on ne retient pas les lieux hors France
    if article.find('./localisationpa') is not None:
        continue
    vedettes = article.xpath('./vedette/i')
    # on ne retient que la première vedette
    vedette = vedettes[0].text if vedettes else ''

    # on ne retient que le premier dpt de localisation
    localisationde = article.find('./localisationde')
    localisationde = localisationde.text if localisationde is not None else 'none'

    # on ne retient que le premier canton de localisation
    localisationca = article.find('./localisationca')
    if localisationca is not None:
        localisationca = re.search(
            r'<localisationca>(.*?)<\/localisationca>',
            etree.tostring(localisationca, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        '''
        localisationca = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca)
    else:
        localisationca = 'none'

    #  on ne retient que la première commune de localisation
    localisationco = article.find('./localisationco')
    if localisationco is not None:
        localisationco = re.search(
            r'<localisationco>(.*?)<\/localisationco>',
            etree.tostring(localisationco, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        - 'c<sup>nes</sup> de '
        - 'c<sup>nes</sup> d’'
            '''
        localisationco = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco)

    else:
        localisationco = 'none'

    # Extraire le contenu textuel de la balise "article"
    article_text = re.sub(r'\W', ' ', article.xpath('string(.)'))

    # on insère tout dans un dictionnaire
    data.append({
        'old-id': article.get('old-id'),
        'vedette': vedette,
        'localisationde': localisationde,
        'localisationca': localisationca,
        'localisationco': localisationco,
        'reference': article_text  # Ajouter la colonne "reference"
    })

df = pd.DataFrame(data)

'''
article : nombre d’articles dans la source => `//article`
article_to_link : nombre d’articles à lier (en France) => `//article[not(localisationpa)]`
article_commune : nombre d’articles de type commune => `//article[not(localisationpa) and not(localisationco)]`
article_loc_com : nombre d’articles localisés dans une commune => `//article[not(localisationpa) and localisationco]`
'''
report = dict(
    article = i,
    article_to_link = len(df.index),
    article_commune = (df.localisationco == 'none').sum(),
    article_loc_com = (df.localisationco != 'none').sum()
)

# check
[print(k,':',v) for k, v in report.items()]
df.iloc[1500:1510, :]

In [ ]:
# Copiez les colonnes d'origine
df['vedette_orig'] = df['vedette']
df['localisationco_orig'] = df['localisationco']

In [ ]:
#premier préprocessing pour les caractères spéciaux

In [ ]:
def replace_special_characters(text):
    text = unidecode(text)
    text = re.sub(r'[-\'\[\]\(\)]', ' ', text)
    return text


df['vedette'] = df['vedette'].apply(replace_special_characters)
df['localisationco'] = df['localisationco'].apply(replace_special_characters)

In [ ]:
df

In [ ]:
# Ajout du dpt code au df
dpt_df = pd.read_csv('../../utils/pouille/resources/departements-region.csv')
dpt_dict = dict(zip(dpt_df['dep_name'], dpt_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['dpt_code'] = df['localisationde'].apply(lambda x: dpt_dict.get(x, 'none'))

#on renomme et réordonne pour plus de clarté.
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune'})
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'reference', 'vedette_orig', 'localisationco_orig']]

df.iloc[1500:1510, :]

In [ ]:
# Chargement du référentiel des communes
fields = ['insee_code', 'DEP_id', 'NCCENR']
main_insee_commune = pd.read_csv("../../utils/pouille/resources/main_insee_commune.tsv",
                                 delimiter='\t',
                                 usecols=fields,
                                 dtype={'insee_code': 'string'})
main_insee_commune['DEP_id'] = main_insee_commune['DEP_id'].apply(lambda x: x[4:])
main_insee_commune.head(5)

In [ ]:
main_insee_commune['NCCENR_orig'] = main_insee_commune['NCCENR']

In [ ]:
main_insee_commune['NCCENR'] = main_insee_commune['NCCENR'].apply(replace_special_characters)

In [ ]:
main_insee_commune.head(5)

In [ ]:
# Liste les départements présents pour mieux filtrer quand pas de localisationde
dpt_list = df['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)

# Exact match methods

## Liage des articles de type commune (exact match mathod 1)

In [ ]:
# dataframe à charger
linked_places_df = pd.DataFrame()

In [ ]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
communes_df.iloc[500:1000, :]
#communes_df.shape

### With dpt

In [ ]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and dpt_code
communes_exact_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette', 'dpt_code'],
                                 right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_dpt_df['insee'] = communes_exact_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_dpt_df = communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_dpt_df = communes_exact_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
communes_exact_dpt_df['method'] = 'dpt_exact'

# Check for duplicate entries
duplicates = communes_exact_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_dpt_df = communes_exact_dpt_df[~duplicates]


# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, communes_exact_dpt_df]).drop_duplicates()

In [ ]:
# Tests
#communes_exact_dpt_df.head(5)
communes_exact_dpt_df

In [ ]:
# des doublons de liage exact match dans un même dpt ?
#communes_exact_dpt_df.article_id.duplicated().sum()
communes_exact_dpt_df.loc[communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

### Without dpt

In [ ]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and NCCENR
communes_exact_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_nodpt_df['insee'] = communes_exact_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_nodpt_df = communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_nodpt_df = communes_exact_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
communes_exact_nodpt_df['method'] = 'nodpt_exact'

duplicates = communes_exact_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_nodpt_df = communes_exact_nodpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, communes_exact_nodpt_df]).drop_duplicates()

In [ ]:
# On cherche à savoir combien ont été liés sans information de département ; cela signifie que dans le XML,
# on a des communes qui ne sont pas localisées.
# Test
communes_exact_nodpt_df
#linked_places_df.shape
communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

In [ ]:
# TODO: on teste quoi ?
communes_exact_nodpt_df.loc[communes_exact_nodpt_df.duplicated(keep=False,subset=['article_id']), :]

## Liage des communes de localisation (exact match method 1)

### With dpt

In [ ]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

In [ ]:
# Merge pas_communes_df and main_insee_commune on localisationco and dpt_code
pas_communes_exact_dpt_df = pd.merge(pas_communes_df,
                                     main_insee_commune,
                                     how='inner',
                                     left_on=['localisationco', 'dpt_code'],
                                     right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_dpt_df['insee'] = pas_communes_exact_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
pas_communes_exact_dpt_df['method'] = 'dpt_exact'

duplicates = pas_communes_exact_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_dpt_df = pas_communes_exact_dpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_dpt_df]).drop_duplicates()


In [ ]:
# Tests
pas_communes_exact_dpt_df.head(5)
#pas_communes_exact_dpt_df.shape
pas_communes_exact_dpt_df.agg(['nunique', 'count', 'size'])

### Without dpt

In [ ]:
import pandas as pd

# Merge pas_communes_df and main_insee_commune on localisationco and NCCENR
pas_communes_exact_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                       main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                       how='inner',
                                       left_on='localisationco',
                                       right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_nodpt_df['insee'] = pas_communes_exact_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
pas_communes_exact_nodpt_df['method'] = 'nodpt_exact'

duplicates = pas_communes_exact_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df[~duplicates]

# Update the linked_places_df dataframe
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_nodpt_df]).drop_duplicates()

In [ ]:
# On cherche à savoir combien ont été liés sans information de département ; cela signifie que dans le XML,
# on a des lieux dans des communes qui ne sont pas localisés.
# Test
pas_communes_exact_nodpt_df
#linked_places_df.shape
pas_communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

## Liage des articles de type commune (exact match method 2)

On cherche à ré-effectuer un deuxième liage en exact match pour tous les articles n'ayant pas été liés dans la première passe. Nous effectuons un pré-processing plus poussé ; on retire tous les mots d'une liste pré-définie par Olivier (tels que Le, La, L', ruisseau, bois, rivière, etc) sur les vedettes du pouillés + les NCCENR du référentiel INSEE puis on ré-applique notre script.

### Data preprocessing

In [ ]:
file_path = '../../utils/pouille/resources/tokens_dicotopo.txt'

with open(file_path, 'r') as file:
    tokens = file.read() 

In [ ]:
# filtrer les mots des noms de lieux
def filter_words(text, tokens):
    text = replace_special_characters(text)  # Appliquer la fonction replace_special_characters
    words = text.split()
    words = [word for word in words if not re.search(r'\b{}\b'.format(re.escape(word.lower())), tokens)]
    filtered_text = ' '.join(words).strip() 
    return filtered_text

df['vedette'] = df['vedette'].apply(lambda x: filter_words(x, tokens=tokens))
df['localisationco'] = df['localisationco'].apply(lambda x: filter_words(x, tokens=tokens))

In [ ]:
main_insee_commune['NCCENR'] = main_insee_commune['NCCENR'].apply(lambda x: filter_words(x, tokens=tokens))

In [ ]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
#communes_df.shape

### with dpt

In [ ]:
import pandas as pd

# Merge communes_df and main_insee_commune on vedette and dpt_code
communes_exact_simple_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette', 'dpt_code'],
                                 right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_simple_dpt_df['insee'] = communes_exact_simple_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_simple_dpt_df = communes_exact_simple_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_simple_dpt_df = communes_exact_simple_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
communes_exact_simple_dpt_df['method'] = 'dpt_exact_simple'

duplicates = communes_exact_simple_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_simple_dpt_df = communes_exact_simple_dpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = communes_exact_simple_dpt_df.loc[communes_exact_simple_dpt_df['method'] == 'dpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

### without dpt

In [ ]:
# Merge communes_df and main_insee_commune on vedette and NCCENR
communes_exact_simple_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
communes_exact_simple_nodpt_df['insee'] = communes_exact_simple_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
communes_exact_simple_nodpt_df['method'] = 'nodpt_exact_simple'

duplicates = communes_exact_simple_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    communes_exact_simple_nodpt_df = communes_exact_simple_nodpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = communes_exact_simple_nodpt_df.loc[communes_exact_simple_nodpt_df['method'] == 'nodpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

## Liage des communes de localisation

In [ ]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

### with dpt

In [ ]:
# Merge pas_communes_df and main_insee_commune on localisationco and dpt_code
pas_communes_exact_simple_dpt_df = pd.merge(pas_communes_df,
                                     main_insee_commune,
                                     how='inner',
                                     left_on=['localisationco', 'dpt_code'],
                                     right_on=['NCCENR', 'DEP_id'])

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_simple_dpt_df['insee'] = pas_communes_exact_simple_dpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'dpt_exact'
pas_communes_exact_simple_dpt_df['method'] = 'dpt_exact_simple'

duplicates = pas_communes_exact_simple_dpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_simple_dpt_df = pas_communes_exact_simple_dpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = pas_communes_exact_simple_dpt_df.loc[pas_communes_exact_simple_dpt_df['method'] == 'dpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

### without dpt

In [ ]:
import pandas as pd

# Merge pas_communes_df and main_insee_commune on localisationco and NCCENR
pas_communes_exact_simple_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                       main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                       how='inner',
                                       left_on='localisationco',
                                       right_on='NCCENR')

# Create a new column 'insee' by combining NCCENR and insee_code
pas_communes_exact_simple_nodpt_df['insee'] = pas_communes_exact_simple_nodpt_df.apply(lambda row: f"{row['NCCENR']} ({row['insee_code']})", axis=1)

# Drop unnecessary columns
pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df.drop(columns=['DEP_id', 'NCCENR', 'insee_code'], axis=1)
pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df.rename(columns={'insee': 'insee_code'})

# Add a 'method' column with value 'nodpt_exact'
pas_communes_exact_simple_nodpt_df['method'] = 'nodpt_exact_simple'

duplicates = pas_communes_exact_simple_nodpt_df.duplicated(['article_id'], keep=False)
if duplicates.any():
    pas_communes_exact_simple_nodpt_df = pas_communes_exact_simple_nodpt_df[~duplicates]

# Filtrer les lignes de communes_exact_simple_dpt_df avec la méthode "dpt_exact_simple"
filtered_df = pas_communes_exact_simple_nodpt_df.loc[pas_communes_exact_simple_nodpt_df['method'] == 'nodpt_exact_simple']

# Filtrer les article_id qui ne sont pas déjà présents dans linked_places_df
filtered_df = filtered_df[~filtered_df['article_id'].isin(linked_places_df['article_id'])]

# Ajouter les lignes filtrées à linked_places_df
linked_places_df = pd.concat([linked_places_df, filtered_df]).drop_duplicates()

linked_places_df

# Fuzzy match method

## Liage des articles de type commune (fuzzy method)

### With dpt

In [ ]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les communes à lier avec le département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' n'est pas égale à 'none'.
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']

# Utiliser la colonne 'vedette' comme valeur pour la colonne 'insee_code'
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']

# Pour chaque ligne dans le DataFrame "communes_fuzzy_dpt_df"
for _, x in communes_fuzzy_dpt_df.iterrows():
    # Sélectionner les valeurs de la colonne 'NCCENR' du DataFrame "main_insee_commune" où la colonne 'DEP_id' correspond à la valeur de la colonne 'dpt_code' de la ligne en cours (x).
    dpt_nccenr = main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR']
    
    # Effectuer un fuzzy match entre les NCCENR du département et la valeur de la colonne 'vedette' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in dpt_nccenr
        if all(word in nccenr.split() for word in x['vedette'].split()) and fuzz.token_sort_ratio(nccenr, x['vedette']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    communes_fuzzy_dpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "communes_fuzzy_dpt_df" et définir toutes ses valeurs comme 'dpt_fuzzy'.
communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "communes_fuzzy_dpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_dpt_df]).drop_duplicates()

In [ ]:
#méthode avec difflib

'''
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['vedette'],
            main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['vedette'].split())
    ],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_dpt_df]).drop_duplicates()
'''

In [ ]:
# Tests
#communes_fuzzy_dpt_df.head(20)
#linked_places_df.shape
#linked_places_df
communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

In [ ]:
communes_fuzzy_dpt_df

### Without dpt

In [ ]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les communes sans département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' est égale à 'none'.
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']

# Utiliser la colonne 'vedette' comme valeur pour la colonne 'insee_code'
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']

# Pour chaque ligne dans le DataFrame "communes_fuzzy_nodpt_df"
for _, x in communes_fuzzy_nodpt_df.iterrows():
    # Effectuer un fuzzy match entre les NCCENR du DataFrame "main_insee_commune" et la valeur de la colonne 'vedette' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in main_insee_commune['NCCENR']
        if all(word in nccenr.split() for word in x['vedette'].split()) and fuzz.token_sort_ratio(nccenr, x['vedette']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    communes_fuzzy_nodpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "communes_fuzzy_nodpt_df" et définir toutes ses valeurs comme 'nodpt_fuzzy'.
communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "communes_fuzzy_nodpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_nodpt_df]).drop_duplicates()

In [ ]:
#liage avec difflib

'''
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['vedette'],
            main_insee_commune['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['vedette'].split())
    ],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_nodpt_df]).drop_duplicates()
'''

In [ ]:
# Tests
communes_fuzzy_nodpt_df
#communes_fuzzy_nodpt_df.shape
#linked_places_df.shape
communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

In [ ]:
communes_fuzzy_nodpt_df

## Liage des communes de localisation (fuzzy method)

### With dpt

In [ ]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les lieux dans les communes avec département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "pas_communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' n'est pas égale à 'none'.
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']

# Utiliser la colonne 'localisationco' comme valeur pour la colonne 'insee_code'
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']

# Pour chaque ligne dans le DataFrame "pas_communes_fuzzy_dpt_df"
for _, x in pas_communes_fuzzy_dpt_df.iterrows():
    # Sélectionner les valeurs de la colonne 'NCCENR' du DataFrame "main_insee_commune" où la colonne 'DEP_id' correspond à la valeur de la colonne 'dpt_code' de la ligne en cours (x).
    dpt_nccenr = main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR']
    
    # Effectuer un fuzzy match entre les NCCENR du département et la valeur de la colonne 'localisationco' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in dpt_nccenr
        if all(word in nccenr.split() for word in x['localisationco'].split()) and fuzz.token_sort_ratio(nccenr, x['localisationco']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    pas_communes_fuzzy_dpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "pas_communes_fuzzy_dpt_df" et définir toutes ses valeurs comme 'dpt_fuzzy'.
pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "pas_communes_fuzzy_dpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()

In [ ]:
#liage avec difflib

'''

import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['localisationco'],
            main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['localisationco'].split())
    ],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()
'''

In [ ]:
# Tests
pas_communes_fuzzy_dpt_df
#linked_places_df.shape
#linked_places_df
pas_communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

In [ ]:
pas_communes_fuzzy_dpt_df

### Without dpt

In [ ]:
import pandas as pd

# Importer la fonction "fuzz" du module "thefuzz"
from thefuzz import fuzz

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
# À partir du DataFrame "main_insee_commune", utiliser la colonne 'NCCENR' comme index et la colonne 'insee_code' comme valeurs pour créer le dictionnaire.
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Filtrer les lieux dans les communes sans département de localisation en utilisant des correspondances floues (fuzzy match)
# Sélectionner les lignes du DataFrame "pas_communes_df" qui ne sont pas présentes dans la colonne 'article_id' du DataFrame "linked_places_df"
# et dont la valeur de la colonne 'dpt_code' est égale à 'none'.
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']

# Utiliser la colonne 'localisationco' comme valeur pour la colonne 'insee_code'
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']

# Pour chaque ligne dans le DataFrame "pas_communes_fuzzy_nodpt_df"
for _, x in pas_communes_fuzzy_nodpt_df.iterrows():
    # Effectuer un fuzzy match entre les NCCENR du DataFrame "main_insee_commune" et la valeur de la colonne 'localisationco' de la ligne en cours (x).
    # Sélectionner les trois meilleures correspondances (matches) ayant une similarité de 10 ou plus (selon fuzz.token_sort_ratio).
    matches = [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in main_insee_commune['NCCENR']
        if all(word in nccenr.split() for word in x['localisationco'].split()) and fuzz.token_sort_ratio(nccenr, x['localisationco']) >= 10
    ][:3]

    # Mettre à jour la colonne 'insee_code' de la ligne en cours (x) avec les trois meilleures correspondances (matches).
    pas_communes_fuzzy_nodpt_df.at[x.name, 'insee_code'] = matches

# Filtrer les communes qui ont été liées avec succès (c'est-à-dire celles ayant au moins une correspondance dans la colonne 'insee_code')
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule, en les séparant par des virgules.
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

# Ajouter une colonne 'method' au DataFrame "pas_communes_fuzzy_nodpt_df" et définir toutes ses valeurs comme 'nodpt_fuzzy'.
pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Mettre à jour le DataFrame "linked_places_df" en y ajoutant les lignes du DataFrame "pas_communes_fuzzy_nodpt_df"
# Enlever les éventuelles lignes dupliquées.
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()

In [ ]:
'''
#liage avec difflib

import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df.apply(
    lambda x: [
        f"{nccenr} ({nccenr_to_insee.get(nccenr, '')}) - {main_insee_commune[main_insee_commune['NCCENR'] == nccenr]['NCCENR_orig'].iloc[0]}"
        for nccenr in difflib.get_close_matches(
            x['localisationco'],
            main_insee_commune['NCCENR'],
            n=3, cutoff=0.1
        )
        if all(word in nccenr.split() for word in x['localisationco'].split())
    ],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()
'''

In [ ]:
pas_communes_fuzzy_nodpt_df

In [ ]:
# Tests
pas_communes_fuzzy_nodpt_df
#linked_places_df.shape
pas_communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

# Postprocessing

## Rapport et analyse

In [ ]:
# On crée un DataFrame new_rows_df en filtrant les lignes de df qui ne sont pas présentes dans linked_places_df
new_rows_df = df[~df['article_id'].isin(linked_places_df['article_id'])]

# On vérifie si 'method' n'est pas dans les colonnes de new_rows_df
if 'method' not in new_rows_df.columns:
    new_rows_df['method'] = 'nulle'

# On utilise la méthode .concat pour ajouter les lignes de new_rows_df à linked_places_df
linked_places_df = pd.concat([linked_places_df, new_rows_df], ignore_index=True)

In [ ]:
linked_places_df

In [ ]:
linked_places_df.agg(['nunique', 'count', 'size'])

In [ ]:
# des doublons?
nombre_de_doublons = linked_places_df.duplicated(subset=['article_id']).sum()
print(nombre_de_doublons)


In [ ]:
linked_places_df.loc[linked_places_df.duplicated(keep=False,subset=['article_id']), :]

In [ ]:
method_counts = linked_places_df['method'].value_counts()
print(method_counts)

In [ ]:
vedette_list = pd.unique(linked_places_df['vedette'])
linked_places_df['sort_key'] = linked_places_df.apply(lambda x: x['localisationco'] if x['localisationco'] in vedette_list else x['vedette'], axis=1)
linked_places_df.sort_values(['method', 'sort_key', 'article_id'], inplace=True)
linked_places_df = linked_places_df.drop('sort_key', axis=1)
linked_places_df = linked_places_df[['article_id', 'vedette', 'vedette_orig', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'localisationco_orig', 'NCCENR_orig', 'insee_code', 'method', 'reference']]
linked_places_df

In [ ]:
empty_rows = linked_places_df[linked_places_df['vedette'] == '']
empty_rows

In [ ]:
empty_rows.agg(['nunique', 'count', 'size'])

## Export

Suite à tous nos traitements, nous exportons le résultat sous la forme d'un fichier CSV. 

Il prend cette forme;

article_id: id du pouillé

vedette: vedette du pouillé, traitée (donc sans ponctuation ni mots jugés 'dérangeants' au liage)

vedette_orig: vedette au format originel

localisationde: département

dpt_code: code du département

canton_code: canton

localisationco: pour les lieux dans des communes, localisationco du pouillé, traité (donc sans ponctuation ni mots jugés 'dérangeants' au liage)

localisationco_orig: localisationco au format originel

nccenr_orig: le nccenr originel du référentiel insee (uniquement pour les exact matches)

insee_code: le code insee. Pour les fuzzy, il y a le nccenr originel + nettoyé + le code insee pour chaque proposition; pour faciliter la correction d'Olivier

method: la méthode effectuée. Il y en a 7 en principe (à part si nous avons une absence de résultat pour la méthode): dpt_exact, nodpt_exact (methode exact match 1), dpt_exact_simple, no_dpt_exact_simple (method exact match 2), dpt_fuzzy, nodpt_fuzzy, nulle (lorsque nous n'avons pas pu faire le liage)

reference: tout le texte de chaque article pouillé


In [ ]:
# Sauvegarder le dataframe fusionné
linked_places_df.to_csv('../../utils/pouille/out/linking_out/liage_po11.csv', index=False)


# Note – réflexions sur le fuzzy

Après avoir effectué des tests, on se rend compte qu'il y a plusieurs incohérences.

1. Difflib, avec main_insee_commune, n'arrive pas à lier certains cas en fuzzy (par exemple, Deneuille & Deneuille-lès-Chantelle). Pourtant, difflib y arrive lorsqu'il utilise un fichier txt de base. Peut-être un souci au niveau de main_insee_commune ? A verifier.

2. Au contraire, TheFuzz arrive à lier dans les deux cas: super. Par contre, il lie des choses en plus (Hures) avec main_insee_commune, mais pas avec le fichier txt. A verifier sur main_insee_commune: il ne semble pourtant pas y avoir d'espace ou de lettres en plus ou en moins.

Pour l'instant, nous gardons thefuzz, mais nous sauvegardons tous les codes de difflib en bas du notebook au cas où, comme les deux manières de faire semblent un peu faillibles (en attendant de comprendre).